In [ ]:
import csv
import sys
import os
import requests 
import random # Imported for randomization
from pptx import Presentation
from pptx.util import Cm, Pt
from pptx.enum.text import PP_ALIGN, MSO_ANCHOR
from pptx.dml.color import RGBColor
from pptx.oxml.xmlchemy import OxmlElement

# --- CONFIGURATION ---
KEYS_SONG = ["Name of Song", "Name of Medley"]
KEYS_SERIES = ["Name of Series/Franchise"]
KEYS_MEDLEY = ["List of all songs to be included in the medley"]
KEYS_REASON = ["Give us up to 30 words (to be put on the slide) for why AVGE should play this song."]
KEYS_LINK = []
KEYS_CATEGORY = ["Anime or Video Game"]

BG_IMAGE_URL = "https://i.pximg.net/img-master/img/2025/08/30/00/01/28/134477717_p0_master1200.jpg"
TEMP_BG_PATH = "temp_slide_bg.jpg"
BOX_TRANSPARENCY = 35 # 65% Opacity

# --- HELPER: Font Size Calculator ---
def get_title_font_size(text, has_medley=False):
    length = len(text)
    base_size = 50 if length < 20 else (36 if length < 40 else 28)
    if has_medley:
        return Pt(base_size * 0.9) 
    return Pt(base_size)

def get_reason_font_size(text, box_height_cm):
    norm_text = text.replace('\\n', '\n').replace('\r\n', '\n').replace('\r', '\n')
    line_count = len(norm_text.split('\n'))
    char_count = len(text)

    if box_height_cm < 6.0:
        if line_count >= 10: size_by_lines = Pt(11)
        elif line_count >= 8: size_by_lines = Pt(12)
        elif line_count >= 6: size_by_lines = Pt(14)
        elif line_count >= 4: size_by_lines = Pt(18)
        else: size_by_lines = Pt(22)
    else:
        if line_count >= 12: size_by_lines = Pt(12)
        elif line_count >= 9: size_by_lines = Pt(14)
        elif line_count >= 7: size_by_lines = Pt(18)
        elif line_count >= 5: size_by_lines = Pt(22)
        else: size_by_lines = Pt(26)

    if char_count > 320: size_by_chars = Pt(14)
    elif char_count > 220: size_by_chars = Pt(18)
    elif char_count > 140: size_by_chars = Pt(22)
    else: size_by_chars = Pt(26)

    return min(size_by_lines, size_by_chars)

# --- HELPER: Image & Transparency ---
def download_bg_image(url, local_path):
    if os.path.exists(local_path): return True
    print(f"Attempting to download background image...")
    headers = {'Referer': 'https://www.pixiv.net/', 'User-Agent': 'Mozilla/5.0'}
    try:
        response = requests.get(url, headers=headers, stream=True, timeout=10)
        response.raise_for_status()
        with open(local_path, 'wb') as out_file:
            out_file.write(response.content)
        return True
    except Exception as e:
        print(f"Warning: Could not download background image. Error: {e}")
        return False

def set_shape_transparency(shape, transparency_percent):
    fill = shape.fill
    fill.solid()
    fill.fore_color.rgb = RGBColor(0, 0, 0)
    alpha_value = int((100 - transparency_percent) * 1000)
    try:
        srgbClr = shape.element.spPr.solidFill.srgbClr
        for child in list(srgbClr):
            if child.tag.endswith("alpha"):
                srgbClr.remove(child)
        alpha = OxmlElement('a:alpha')
        alpha.set('val', str(alpha_value))
        srgbClr.append(alpha)
    except AttributeError:
         pass 

# --- HELPER: Data Extraction ---
def get_indices_for_keys(header_row, keys):
    indices = []
    clean_keys = [k.lower().strip() for k in keys]
    for i, column_name in enumerate(header_row):
        if column_name.lower().strip() in clean_keys:
            indices.append(i)
    return indices

def get_first_non_empty(row, indices):
    for i in indices:
        if i < len(row):
            val = row[i].strip()
            if val:
                return val
    return ""

# --- SLIDE CREATION FUNCTIONS ---

def create_section_title_slide(prs, title_text, bg_ready):
    """Creates a slide with just the Category Title."""
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    if bg_ready:
        slide.shapes.add_picture(TEMP_BG_PATH, 0, 0, width=prs.slide_width, height=prs.slide_height)

    # Large Center Box
    box_w = Cm(24)
    box_h = Cm(6)
    box_x = (prs.slide_width - box_w) / 2
    box_y = (prs.slide_height - box_h) / 2
    
    shape = slide.shapes.add_shape(1, box_x, box_y, box_w, box_h)
    set_shape_transparency(shape, BOX_TRANSPARENCY)
    shape.line.fill.background()

    tf = shape.text_frame
    tf.vertical_anchor = MSO_ANCHOR.MIDDLE
    p = tf.paragraphs[0]
    p.alignment = PP_ALIGN.CENTER
    run = p.add_run()
    run.text = title_text
    run.font.name = "Merriweather"
    run.font.size = Pt(80)
    run.font.bold = True
    run.font.color.rgb = RGBColor(255, 255, 255)

def create_content_slide(prs, data_row, bg_ready):
    """Creates the standard 3-box detail slide."""
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    if bg_ready:
        slide.shapes.add_picture(TEMP_BG_PATH, 0, 0, width=prs.slide_width, height=prs.slide_height)

    white = RGBColor(255, 255, 255)
    
    # Extract Data
    song_val = data_row['song']
    series_val = data_row['series']
    medley_val = data_row['medley']
    reason_val = data_row['reason']
    link_val = data_row['link']

    # Layout Constants
    BOX_LEFT = Cm(2.0)
    BOX_WIDTH = Cm(29.8)
    TOP_START_Y = Cm(1.5)
    GAP = Cm(1.0)
    
    # Dynamic Logic
    if medley_val:
        TOP_HEIGHT = Cm(6.0)
        MID_HEIGHT = Cm(5.5)
    else:
        TOP_HEIGHT = Cm(4.5)
        MID_HEIGHT = Cm(7.0)

    MID_START_Y = TOP_START_Y + TOP_HEIGHT + GAP
    BOT_START_Y = MID_START_Y + MID_HEIGHT + GAP
    BOT_HEIGHT = Cm(2.5)

    # --- 1. TOP BOX ---
    top_box = slide.shapes.add_shape(1, BOX_LEFT, TOP_START_Y, BOX_WIDTH, TOP_HEIGHT)
    set_shape_transparency(top_box, BOX_TRANSPARENCY) 
    top_box.line.fill.background() 

    tf = top_box.text_frame
    tf.word_wrap = True
    tf.vertical_anchor = MSO_ANCHOR.MIDDLE
    tf.clear()

    title_text = song_val if song_val else "Unknown Song"
    title_size = get_title_font_size(title_text, has_medley=bool(medley_val))

    p = tf.paragraphs[0]
    p.alignment = PP_ALIGN.CENTER
    run = p.add_run()
    run.text = title_text
    run.font.name = "Merriweather" 
    run.font.size = title_size  
    run.font.bold = True
    run.font.color.rgb = white

    if series_val:
        p_sub = tf.add_paragraph()
        p_sub.alignment = PP_ALIGN.CENTER
        p_sub.space_before = Pt(5)
        run_sub = p_sub.add_run()
        run_sub.text = series_val
        run_sub.font.name = "Merriweather"
        run_sub.font.size = Pt(title_size.pt * 0.6) 
        run_sub.font.bold = True
        run_sub.font.color.rgb = white

    if medley_val:
        p_med = tf.add_paragraph()
        p_med.alignment = PP_ALIGN.CENTER
        p_med.space_before = Pt(5)
        run_med = p_med.add_run()
        run_med.text = "Includes: " + medley_val
        run_med.font.name = "Merriweather"
        run_med.font.size = Pt(title_size.pt * 0.45)
        run_med.font.color.rgb = white

    # --- 2. MIDDLE BOX ---
    mid_box = slide.shapes.add_shape(1, BOX_LEFT, MID_START_Y, BOX_WIDTH, MID_HEIGHT)
    set_shape_transparency(mid_box, BOX_TRANSPARENCY) 
    mid_box.line.fill.background()

    tf_mid = mid_box.text_frame
    tf_mid.word_wrap = True
    tf_mid.vertical_anchor = MSO_ANCHOR.TOP
    tf_mid.margin_top = Cm(0.5)
    tf_mid.clear()

    if reason_val:
        normalized_text = reason_val.replace('\\n', '\n').replace('\r\n', '\n').replace('\r', '\n')
        reason_size = get_reason_font_size(normalized_text, MID_HEIGHT.cm)
        lines = [line for line in normalized_text.split('\n') if line.strip()]

        for i, line in enumerate(lines):
            line_content = line.strip()
            if i == 0:
                p_reason = tf_mid.paragraphs[0]
            else:
                p_reason = tf_mid.add_paragraph()
                p_reason.space_before = Pt(6) 
            p_reason.alignment = PP_ALIGN.CENTER
            run_reason = p_reason.add_run()
            run_reason.text = line_content
            run_reason.font.name = "Merriweather"
            run_reason.font.size = reason_size
            run_reason.font.color.rgb = white

    # --- 3. BOTTOM BOX ---
    bot_box = slide.shapes.add_shape(1, BOX_LEFT, BOT_START_Y, BOX_WIDTH, BOT_HEIGHT)
    set_shape_transparency(bot_box, BOX_TRANSPARENCY) 
    bot_box.line.fill.background()

    tf_bot = bot_box.text_frame
    tf_bot.word_wrap = True
    tf_bot.vertical_anchor = MSO_ANCHOR.MIDDLE
    tf_bot.clear()

    p_link = tf_bot.paragraphs[0]
    p_link.alignment = PP_ALIGN.CENTER
    
    link_text = link_val if link_val else "N/A"
    link_size = Pt(18) if len(link_text) < 50 else Pt(14)

    run_label = p_link.add_run()
    run_label.text = "Link: "
    run_label.font.name = "Merriweather"
    run_label.font.size = link_size
    run_label.font.bold = True
    run_label.font.color.rgb = white
    
    run_link = p_link.add_run()
    run_link.text = link_text
    run_link.font.name = "Merriweather"
    run_link.font.size = link_size
    run_link.font.color.rgb = white

# --- MAIN ORCHESTRATOR ---
def create_final_slideshow(csv_filename, output_filename):
    bg_image_ready = download_bg_image(BG_IMAGE_URL, TEMP_BG_PATH)

    print("Initializing PowerPoint...")
    prs = Presentation()
    prs.slide_width = Cm(33.867)
    prs.slide_height = Cm(19.05)

    anime_list = []
    game_list = []
    
    try:
        print(f"Reading {csv_filename}...")
        with open(csv_filename, mode='r', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile, delimiter=',')
            headers = next(reader, None)
            if not headers: return

            # Get Indices
            idxs_song = get_indices_for_keys(headers, KEYS_SONG)
            idxs_series = get_indices_for_keys(headers, KEYS_SERIES)
            idxs_medley = get_indices_for_keys(headers, KEYS_MEDLEY)
            idxs_reason = get_indices_for_keys(headers, KEYS_REASON)
            idxs_link = get_indices_for_keys(headers, KEYS_LINK)
            idxs_cat = get_indices_for_keys(headers, KEYS_CATEGORY)

            # Parse Data
            for row in reader:
                data = {
                    'song': get_first_non_empty(row, idxs_song),
                    'series': get_first_non_empty(row, idxs_series),
                    'medley': get_first_non_empty(row, idxs_medley),
                    'reason': get_first_non_empty(row, idxs_reason),
                    'link': get_first_non_empty(row, idxs_link),
                    'category': get_first_non_empty(row, idxs_cat)
                }

                # Simple Validation
                if not data['song'] and not data['series']:
                    continue

                # Sort into lists
                cat = data['category'].lower()
                if "anime" in cat:
                    anime_list.append(data)
                elif "game" in cat or "video" in cat:
                    game_list.append(data)
                else:
                    # Default/Fallback: treat as game if unknown, or handle as you wish
                    # For now, let's append to game list or maybe just skip? 
                    # Let's add to game list as fallback.
                    game_list.append(data)

        # Randomize
        print(f"Randomizing {len(anime_list)} Anime songs and {len(game_list)} Video Game songs...")
        random.shuffle(anime_list)
        random.shuffle(game_list)

        # Generate Slides
        # 1. Anime Section
        if anime_list:
            create_section_title_slide(prs, "Anime", bg_image_ready)
            for item in anime_list:
                create_content_slide(prs, item, bg_image_ready)
        
        # 2. Video Game Section
        if game_list:
            create_section_title_slide(prs, "Video Game", bg_image_ready)
            for item in game_list:
                create_content_slide(prs, item, bg_image_ready)

        prs.save(output_filename)
        print(f"Success! Saved to {output_filename}")

    except Exception as e:
        import traceback
        traceback.print_exc()
    finally:
        if os.path.exists(TEMP_BG_PATH):
            try:
                os.remove(TEMP_BG_PATH)
            except:
                pass

if __name__ == "__main__":
    dummy_file = "Insert your CSV file here"
    if not os.path.exists(dummy_file):
        print(f"Generating dummy CSV: {dummy_file}")
        with open(dummy_file, "w", newline='', encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(["Name of Song", "Name of Series/Franchise", "Anime or Video Game", "List of all songs to be included in the medley", "Reason", "Link"])
            
            writer.writerow(["Anime Song 1", "Naruto", "Anime", "", "Classic anime opening.", "http://a.com"])
            writer.writerow(["Anime Song 2", "Bleach", "Anime", "", "Another classic.", "http://b.com"])
            writer.writerow(["Game Song 1", "Mario", "Video Game", "", "It's a me.", "http://c.com"])
            writer.writerow(["Medley Game", "Zelda", "Video Game", "Saria's Song, Zelda's Lullaby", "Ocarina time.", "http://d.com"])
            writer.writerow(["Anime Song 3", "One Piece", "Anime", "", "Pirate king.", "http://e.com"])

    create_final_slideshow(dummy_file, "Insert your path here")

Attempting to download background image...
Initializing PowerPoint...
Reading C:\Users\h8kur\Downloads\tsv files\songvoting.csv...
Randomizing 16 Anime songs and 23 Video Game songs...
Success! Saved to C:\Users\h8kur\Downloads\tsv files\avgeslides21.pptx
